In [1]:
import pandas as pd 
from collections import Counter, defaultdict
from tqdm import tqdm
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_error, mean_absolute_error

In [2]:
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorWithPadding, AutoTokenizer, AutoModelForSequenceClassification
import torch
from datasets import Dataset, DatasetDict, load_dataset
import evaluate


2023-04-12 21:36:42.132950: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [52]:
train = pd.read_csv("jigsaw-train.csv")
dev = pd.read_csv("jigsaw-dev.csv")
train = train.rename({"comment_text": "text", "toxic": "label"}, axis=1).drop("id", axis=1)
dev = dev.rename({"comment_text": "text", "toxic": "label"}, axis=1).drop("id", axis=1)
train["label"] = train["label"].astype(int)
dev["label"] = dev["label"].astype(int)


In [53]:
train_ds = Dataset.from_pandas(train)
dev_ds = Dataset.from_pandas(dev)

In [54]:
model_name = "microsoft/MiniLM-L12-H384-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=128, padding="max_length")

tokenized_train = train_ds.map(preprocess_function, batched=True)
tokenized_dev = dev_ds.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/159571 [00:00<?, ? examples/s]

Map:   0%|          | 0/31969 [00:00<?, ? examples/s]

In [57]:
output_dir = '.'#output to current directory



def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    learning_rate=2e-5,
    num_train_epochs=2,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,    
    weight_decay=0.01,
    do_eval=True,
    seed=1234,
    save_strategy='epoch',
    evaluation_strategy='epoch',
    logging_dir=output_dir + 'logs/',
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)


trainer = Trainer(
    model_init=model_init,    
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_dev,    
    tokenizer=tokenizer,
    data_collator=data_collator,
#     compute_metrics=compute_metrics_for_regression
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 